# QSIRecon Basics: Diffusion MRI Reconstruction

This notebook demonstrates how to use the QSIRecon runner for diffusion reconstruction and connectivity analysis.

## Overview

QSIRecon performs:
- Diffusion model fitting (DTI, CSD, MAPMRI, etc.)
- Tractography
- Structural connectivity matrix generation
- Regional quantification

## Prerequisites

- Docker installed and running
- QSIPrep preprocessed data
- Reconstruction specification file (JSON/YAML)
- FreeSurfer license file

## Setup

In [1]:
from pathlib import Path
from voxelops import (
    run_qsirecon,
    QSIReconInputs,
    QSIReconDefaults,
)
import json

## Define Paths

In [5]:
# Input paths
qsiprep_dir = Path("/media/storage/yalab-dev/qsiprep_test/qsiprep_output/")
participant = "01"
recon_spec = Path("/home/galkepler/Projects/yalab-devops/VoxelOps/qsirecon_spec.yaml")
fs_license = Path("/home/galkepler/misc/freesurfer/license.txt")

# Output paths (optional)
output_dir = Path("/media/storage/yalab-dev/qsiprep_test/qsirecon_output/")
work_dir = Path("/media/storage/yalab-dev/qsiprep_test/work/qsirecon/")

# Datasets to include
datasets = {
    "atlases": "/media/storage/yalab-dev/voxelops/Schaefer2018Tian2020_atlases"
}

## Understanding Reconstruction Specs

QSIRecon requires a reconstruction specification file that defines:
- Which diffusion models to fit
- Tractography algorithms to use
- Parcellation schemes for connectivity
- Atlas registrations

Common specs:
- `dsi_studio_gqi`: DSI Studio's GQI model with tractography
- `mrtrix_multishell_msmt_ACT-fast`: Multi-shell MSMT-CSD with ACT
- `amico_noddi`: AMICO NODDI model fitting
- `dipy_mapmri`: DIPY MAPMRI reconstruction

In [6]:
# View reconstruction spec (yaml file)
import yaml

if recon_spec.exists():
    spec_content = yaml.safe_load(recon_spec.read_text())
    print("Reconstruction Specification:")
    print(yaml.dump(spec_content, indent=2))
else:
    print(f"Recon spec not found: {recon_spec}")

Reconstruction Specification:
name: gal_multishell_scalars
nodes:
- action: DKI_reconstruction
  name: dipy_dki
  parameters:
    wmti: true
    write_fibgz: false
    write_mif: false
  qsirecon_suffix: DIPYDKI
  software: Dipy
- action: fit_noddi
  name: amico_noddi
  parameters:
    dIso: 0.003
    dPar: 0.0017
    isExvivo: false
  qsirecon_suffix: AMICONODDI
  software: AMICO
- action: MAPMRI_reconstruction
  name: dipy_mapmri
  parameters:
    anisotropic_scaling: false
    big_delta: null
    bval_threshold: 2000
    dti_scale_estimation: false
    laplacian_regularization: true
    laplacian_weighting: 0.2
    radial_order: 6
    small_delta: null
    write_fibgz: true
    write_mif: true
  qsirecon_suffix: DIPYMAPMRI
  software: Dipy
- action: reconstruction
  input: qsirecon
  name: dsistudio_gqi
  parameters:
    method: gqi
  qsirecon_suffix: DSIStudio
  software: DSI Studio
- action: export
  input: dsistudio_gqi
  name: scalar_export
  qsirecon_suffix: DSIStudio
  softwar

## Basic Usage

### Option 1: Use Default Configuration

In [7]:
# Create inputs
inputs = QSIReconInputs(
    qsiprep_dir=qsiprep_dir,
    participant=participant,
    recon_spec=recon_spec,
    output_dir=output_dir,
    work_dir=work_dir,
    datasets=datasets,
    atlases=['Schaefer2018N100n7Tian2020S1',
 'Schaefer2018N100n7Tian2020S2',
 'Schaefer2018N100n7Tian2020S3',
 'Schaefer2018N100n7Tian2020S4',
 'Schaefer2018N200n7Tian2020S1',
 'Schaefer2018N200n7Tian2020S2',
 'Schaefer2018N200n7Tian2020S3',
 'Schaefer2018N200n7Tian2020S4',
 'Schaefer2018N300n7Tian2020S1',
 'Schaefer2018N300n7Tian2020S2',
 'Schaefer2018N300n7Tian2020S3',
 'Schaefer2018N300n7Tian2020S4',
 'Schaefer2018N400n7Tian2020S1',
 'Schaefer2018N400n7Tian2020S2',
 'Schaefer2018N400n7Tian2020S3',
 'Schaefer2018N400n7Tian2020S4',
 'Schaefer2018N500n7Tian2020S1',
 'Schaefer2018N500n7Tian2020S2',
 'Schaefer2018N500n7Tian2020S3',
 'Schaefer2018N500n7Tian2020S4',
 'Schaefer2018N600n7Tian2020S1',
 'Schaefer2018N600n7Tian2020S2',
 'Schaefer2018N600n7Tian2020S3',
 'Schaefer2018N600n7Tian2020S4',
 'Schaefer2018N800n7Tian2020S1',
 'Schaefer2018N800n7Tian2020S2',
 'Schaefer2018N800n7Tian2020S3',
 'Schaefer2018N800n7Tian2020S4',
 'Schaefer2018N900n7Tian2020S1',
 'Schaefer2018N900n7Tian2020S2',
 'Schaefer2018N900n7Tian2020S3',
 'Schaefer2018N900n7Tian2020S4',
 'Schaefer2018N1000n7Tian2020S1',
 'Schaefer2018N1000n7Tian2020S2',
 'Schaefer2018N1000n7Tian2020S3',
 'Schaefer2018N1000n7Tian2020S4']
)

# Run with defaults
result = run_qsirecon(
    inputs, fs_license=fs_license, docker_image="pennlinc/qsirecon:1.1.1"
)

print(f"Success: {result['success']}")
print(f"Duration: {result['duration_human']}")


Running qsirecon for participant 01
Command: docker run -it --rm --user 1000:1000 -v /media/storage/yalab-dev/qsiprep_test/qsiprep_output:/data:ro -v /media/storage/yalab-dev/qsiprep_test/qsirecon_output:/out -v /media/storage/yalab-dev/qsiprep_test/work/qsirecon:/work -v /home/galkepler/Projects/yalab-devops/VoxelOps/qsirecon_spec.yaml:/recon_spec.yaml:ro -v /home/galkepler/misc/freesurfer/license.txt:/license.txt:ro -v /media/storage/yalab-dev/voxelops/Schaefer2018Tian2020_atlases:/datasets/atlases:ro pennlinc/qsirecon:1.1.1 /data /out participant --participant-label 01 --nprocs 8 --mem-mb 16000 --work-dir /work --datasets atlases=/datasets/atlases --atlases 4S156Parcels 4S256Parcels 4S356Parcels 4S456Parcels 4S556Parcels 4S656Parcels 4S756Parcels 4S856Parcels 4S956Parcels 4S1056Parcels AICHA384Ext Brainnetome246Ext AAL116 Gordon333Ext Schaefer2018N100n7Tian2020S1 Schaefer2018N100n7Tian2020S2 Schaefer2018N100n7Tian2020S3 Schaefer2018N100n7Tian2020S4 Schaefer2018N200n7Tian2020S1 Scha

ProcedureExecutionError: qsirecon failed: qsirecon failed with exit code 1

Stderr (last 1000 chars):
the input device is not a TTY


### Option 2: Increase Resources for Tractography

In [ ]:
# Tractography is computationally intensive
result = run_qsirecon(
    inputs,
    fs_license=fs_license,
    nprocs=32,  # More cores = faster tractography
    mem_gb=64,  # Sufficient memory for large atlases
)

print(f"Success: {result['success']}")
print(f"Processing time: {result['duration_human']}")

### Option 3: Custom Configuration

In [ ]:
# Create custom configuration
config = QSIReconDefaults(
    nprocs=24,
    mem_gb=48,
    skip_bids_validation=False,
    fs_license=fs_license,
    docker_image="pennlinc/qsirecon:0.19.1",  # Pin version
)

result = run_qsirecon(inputs, config)

print(f"Success: {result['success']}")

## Inspect Execution Record

In [ ]:
print("Execution Details:")
print(f"  Tool: {result['tool']}")
print(f"  Participant: {result['participant']}")
print(f"  Duration: {result['duration_human']}")
print(f"  Success: {result['success']}")

print("\nConfiguration Used:")
config_used = result["config"]
print(f"  Cores: {config_used.nprocs}")
print(f"  Memory: {config_used.mem_gb}GB")
print(f"  Docker image: {config_used.docker_image}")

## Check Expected Outputs

In [ ]:
outputs = result["expected_outputs"]

print("Expected Output Locations:")
print(f"  QSIRecon directory: {outputs.qsirecon_dir}")
print(f"  Participant directory: {outputs.participant_dir}")
print(f"  HTML report: {outputs.html_report}")
print(f"  Work directory: {outputs.work_dir}")

# Verify outputs
print("\nOutput Validation:")
print(f"  HTML report exists: {outputs.html_report.exists()}")
print(f"  Participant dir exists: {outputs.participant_dir.exists()}")

## Explore Reconstruction Outputs

In [ ]:
if outputs.participant_dir.exists():
    print(f"Reconstruction outputs for {participant}:\n")

    # Organize by type
    scalar_maps = []
    tractography = []
    connectivity = []
    other = []

    for f in outputs.participant_dir.rglob("*"):
        if f.is_file():
            name = f.name
            if name.endswith(".nii.gz"):
                scalar_maps.append(f)
            elif name.endswith((".trk", ".tck", ".fib")):
                tractography.append(f)
            elif name.endswith(".csv"):
                connectivity.append(f)
            else:
                other.append(f)

    print(f"Scalar Maps ({len(scalar_maps)}):")
    for f in sorted(scalar_maps)[:10]:  # Show first 10
        print(f"  {f.name}")

    print(f"\nTractography Files ({len(tractography)}):")
    for f in sorted(tractography):
        print(f"  {f.name} ({f.stat().st_size / (1024**2):.1f} MB)")

    print(f"\nConnectivity Matrices ({len(connectivity)}):")
    for f in sorted(connectivity):
        print(f"  {f.name}")
else:
    print("Participant directory not found")

## Load and Visualize Connectivity Matrix

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Find connectivity matrices
connectivity_files = list(outputs.participant_dir.rglob("*connectivity*.csv"))

if connectivity_files:
    # Load first connectivity matrix
    conn_file = connectivity_files[0]
    print(f"Loading: {conn_file.name}\n")

    # Load matrix
    conn_matrix = pd.read_csv(conn_file, index_col=0)

    print(f"Matrix shape: {conn_matrix.shape}")
    print(f"ROIs: {len(conn_matrix)}")
    print(f"\nFirst few ROI names:")
    print(conn_matrix.index[:5].tolist())

    # Visualize
    plt.figure(figsize=(12, 10))
    plt.imshow(conn_matrix.values, cmap="hot", interpolation="nearest")
    plt.colorbar(label="Connection Strength")
    plt.title(f"Structural Connectivity Matrix\n{conn_file.name}")
    plt.xlabel("ROI Index")
    plt.ylabel("ROI Index")
    plt.tight_layout()
    plt.show()

    # Basic statistics
    print(f"\nMatrix Statistics:")
    print(f"  Mean connection strength: {conn_matrix.values.mean():.4f}")
    print(f"  Max connection strength: {conn_matrix.values.max():.4f}")
    print(f"  Sparsity: {(conn_matrix.values == 0).sum() / conn_matrix.size:.1%}")
else:
    print("No connectivity matrices found")

## View HTML QC Report

In [ ]:
from IPython.display import IFrame

if outputs.html_report.exists():
    IFrame(src=str(outputs.html_report), width=900, height=600)
else:
    print(f"HTML report not found: {outputs.html_report}")

## Processing Multiple Reconstruction Specs

You might want to run different reconstruction pipelines on the same data:

In [ ]:
# Define multiple reconstruction specs
recon_specs = [
    Path("/config/recon_specs/dsi_studio_gqi.json"),
    Path("/config/recon_specs/mrtrix_msmt_csd.json"),
    Path("/config/recon_specs/amico_noddi.json"),
]

config = QSIReconDefaults(
    nprocs=24,
    mem_gb=48,
    fs_license=fs_license,
)

results = []

for spec in recon_specs:
    if not spec.exists():
        print(f"Skipping {spec.name} (not found)")
        continue

    print(f"\nRunning reconstruction: {spec.stem}")

    inputs = QSIReconInputs(
        qsiprep_dir=qsiprep_dir,
        participant=participant,
        recon_spec=spec,
    )

    try:
        result = run_qsirecon(inputs, config)
        results.append(result)
        print(f"  ✓ Success in {result['duration_human']}")
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        results.append({"recon_spec": spec.name, "success": False, "error": str(e)})

# Summary
print(f"\nCompleted {len(results)} reconstructions")
for r in results:
    if r.get("success"):
        print(f"  ✓ {r['inputs'].recon_spec.stem}: {r['duration_human']}")
    else:
        print(f"  ✗ {r.get('recon_spec', 'unknown')}: Failed")

## Error Handling

In [ ]:
from voxelops.exceptions import (
    ProcedureExecutionError,
    InputValidationError,
)

try:
    result = run_qsirecon(
        inputs,
        fs_license=fs_license,
        nprocs=24,
    )
    print(f"Success: {result['success']}")

except InputValidationError as e:
    print(f"Input validation failed: {e}")
    print("Common issues:")
    print("  - QSIPrep directory doesn't exist")
    print("  - Participant not found in QSIPrep output")
    print("  - Reconstruction spec file not found or invalid")

except ProcedureExecutionError as e:
    print(f"Execution failed: {e}")
    print(f"Check logs: {result.get('log_file')}")

except Exception as e:
    print(f"Unexpected error: {e}")

## Next Steps

After QSIRecon reconstruction:

1. Review the HTML QC report
2. Inspect connectivity matrices
3. Visualize tractography (using DSI Studio, TrackVis, etc.)
4. Run parcellation with QSIParc (see `04_qsiparc_basics.ipynb`)
5. Perform network analysis on connectivity matrices

## Tips

- **Reconstruction specs**: Choose based on your acquisition (single-shell vs multi-shell)
- **Resources**: Tractography is CPU-intensive - allocate sufficient cores
- **Memory**: Some atlases require significant memory (64GB+ for high-resolution)
- **Validation**: Always review the HTML report for quality control
- **Multiple specs**: You can run different models on the same preprocessed data
- **Connectivity matrices**: Different atlases produce different sized matrices
- **Version pinning**: Use specific Docker versions for reproducibility